In [3]:
import pandas as pd
import psycopg2 as psy
import requests
import datetime
from dotenv import load_dotenv
import os

In [25]:
#Access virtual environment for sensitive details
load_dotenv()
api_key= os.getenv('WEATHER_API')


base_url= 'http://api.weatherstack.com/current'

locations= ['Edmonton', 'Calgary', 'Toronto', 'Vancouver', 'Winnipeg']

In [26]:
# Using API to access data 
weather_data=[]
for location in locations:
    params= {
    'access_key': api_key,
    'query': location
}
    response= requests.get(base_url, params= params)
    
    data= response.json()
    weather_data.append(data)

In [27]:
weather_data

[{'request': {'type': 'City',
   'query': 'Edmonton, Canada',
   'language': 'en',
   'unit': 'm'},
  'location': {'name': 'Edmonton',
   'country': 'Canada',
   'region': 'Alberta',
   'lat': '53.550',
   'lon': '-113.500',
   'timezone_id': 'America/Edmonton',
   'localtime': '2024-06-21 22:30',
   'localtime_epoch': 1719009000,
   'utc_offset': '-6.0'},
  'current': {'observation_time': '04:30 AM',
   'temperature': 18,
   'weather_code': 113,
   'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'],
   'weather_descriptions': ['Clear'],
   'wind_speed': 9,
   'wind_degree': 200,
   'wind_dir': 'SSW',
   'pressure': 1013,
   'precip': 1,
   'humidity': 48,
   'cloudcover': 0,
   'feelslike': 18,
   'uv_index': 3,
   'visibility': 9,
   'is_day': 'no'}},
 {'request': {'type': 'City',
   'query': 'Calgary, Canada',
   'language': 'en',
   'unit': 'm'},
  'location': {'name': 'Calgary',
   'country': 'Canada',
   'region': 'Al

In [33]:
# We need to extract the relevant data from the response
# and convert it into a DataFrame

# Extract the relevant data
cleaned_data = []
for item in weather_data:
    location = item["location"]["name"]
    country = item["location"]["country"]
    temperature = item["current"]["temperature"]
    temp_feels_like= item["current"]["feelslike"]
    wind_speed = item["current"]["wind_speed"]
    pressure = item["current"]["pressure"]
    humidity = item["current"]["humidity"]
    local_time= item["location"]['localtime']
    observation_time = item["current"]["observation_time"]

    cleaned_data.append([location, country, temperature,temp_feels_like, wind_speed, pressure, humidity,local_time, observation_time])
    

In [35]:
# Convert the data into a DataFrame
columns = ["location", "country", "temperature", "temp_feels_like","wind_speed", "pressure", "humidity","local_time", "observation_time"]
df = pd.DataFrame(cleaned_data, columns=columns)


In [36]:
df.head()

,location,country,temperature,temp_feels_like,wind_speed,pressure,humidity,local_time,observation_time
0,Edmonton,Canada,18,18,9,1013,48,2024-06-21 22:30,04:30 AM
1,Calgary,Canada,12,12,11,1017,62,2024-06-21 22:30,04:30 AM
2,Toronto,Canada,20,20,4,1018,88,2024-06-22 00:30,04:30 AM
3,Vancouver,Canada,24,26,7,1011,41,2024-06-21 21:30,04:30 AM
4,Winnipeg,Canada,19,19,4,1017,55,2024-06-21 23:30,04:30 AM


In [ ]:
#Creating connection

conn= psy.connect(
    dbname= os.getenv('DB_NAME'),
    user= os.getenv('DB_USER'),
    host=os.getenv('DB_HOST'),
    password= os.getenv('PASSWORD'),
    port= os.getenv('DB_PORT')
)